TABELAS NECESSÁRIAS:
    Antigas:
        a_campdep.votacao_bancada [codproposicao, tipo, numero, ano, codsessao, datavotacao, sigla, orientacao]

        a_campdep.votacao_deputado [codproposicao,tipo, numero, ano, codsessao, datavotacao, idecadastro, nome,   partido, uf, voto]
    
    Novas:
        desenv.cam_total_votacao_dia [dia, numero_votacoes]
        desenv.cam_total_votacao_mes [mes, numero_votacoes]
        desenv.cam_total_votacao_ano [ano, numero_votacoes]
        camdep.votacao_bancada_partido
        desenv.cam_votacao_bancada_deputado
        
        desenv.votacao_governo_deputados_dia -- tem a votacao de cada deputado em comp com o gov
        desenv.votacao_governo_partidos_dia  -- tem a votacao de cada partido em comp com o gov
        
   Atualiza:
        campdep.votacao_deputado
        
 

##### Metodologia

Algumas considerações metodológicas da construção das tabelas:

- Serão considerados somente o apoio de quando o partido/bloco explicita sua orientação. 
Então, orientação == ['Liberado', 'Abstenção', NaN] serão descartados

- Parlamentares com voto == 'Art. 17' serão descartados pois se referem ao presidente da casa

- Parlamentares com voto == '-' serão descartados pois indicam que o deputado não estava presente

- Somente votos que seguem integralmente a orientação do governo/bloco são considerados como apoio (Sim-Sim, Não-Não, Obstrução-Obstrução). Quaisquer diferenças do sugerido pelo partido/bloco enquadram-se como oposição (Sim-Não, Sim-Obstrução...)

##### Observações

As orientações possíveis são: 'Sim', 'Obstrução', 'Não', 'Liberado', 'Abstenção'

Os votos possíveis são: 'Sim', 'Não', 'Obstrução', 'Abstenção', 'Branco'

In [1]:
partidos = ['PMDB','PTB','PPB','PL','PST','PTR','PRS','PDC','PDS', 'PPR','PRN', 'PAN','PRONA','PTDOB', 'PMR','PFL','SDD', 'SD','PDT','PT','DEM','PCDOB','PSB','PSDB','PTC','PSC','PMN','PRP','PPS','PV','PTdoB','PP','PSTU','PCB','PRTB','PHS','PSDC','PCO','PTN','PSL','PRB','PSOL','PR','PSD','PPL','PEN','PROS','SOLIDARIED','NOVO','REDE','PMB']
blocos   = ['GOV', 'MINORIA', 'MAIORIA', 'APOIO AO GOVERNO']
relacoes = {'SD': ['SDD', 'SOLIDARIED', 'SD'],
           'GOV': ['APOIO AO GOVERNO', 'GOV', 'GOV.'],
           'DEM': ['DEM', 'PFL'],
           'PR': ['PRONA', 'PL', 'PR', 'PST'],
           'PTB': ['PAN', 'PTB'],
           'PRB': ['PRB', 'PMR'],
           'PP': ['PP', 'PPB']}

In [2]:
# GLOBAL
SCHEMA = 'desenv.cam_'

In [44]:
import sqlalchemy
from sqlalchemy.sql import text
import pandas as pd
import glob
import os
import pickle
import numpy as np
import tqdm
import yaml

#### CONECTA AO BANCO

In [45]:
with open('server_config.yaml', 'r') as f:
        server = yaml.load(f)

host = server['host']
database = server['database']
user = server['user']
password = server['password']

url = 'postgresql://{}:{}@{}/{}'
url = url.format(user, password, host, database)
conn = sqlalchemy.create_engine(url)

In [5]:
#### CRIA TABLE DAS SIGLAS ATUALIZADAS

In [6]:
rel = []
for key, val in relacoes.items():
    for v in val:
        rel.append((key, v))
rel = pd.DataFrame(rel, columns=['sigla_atualizada', 'sinonimo'])
rel.to_sql('sigla_partido_atulizada', conn, schema='desenv', if_exists='replace')

#### CRIA TABELA QTDADE VOTACAO DIA
OBS: Preferi usar date_trunc ao invés de criar um field novo porque ainda posso usar as vantagens do tipo timestamp


In [7]:
query = text("""
            DROP TABLE IF EXISTS {0}total_votacao_dia;
            CREATE TABLE {0}total_votacao_dia AS
                SELECT  date_trunc('day', t.datavotacao) as datavotacao_trunc,
                    COUNT(date_trunc('day', t.datavotacao)) as numero_votacao
                FROM (
                    SELECT DISTINCT datavotacao, codproposicao, codsessao
                    FROM a_camdep.votacao_bancada
                ) t
                GROUP BY datavotacao_trunc
                ORDER BY datavotacao_trunc DESC;
            """.format(SCHEMA))
conn.execute(query, autocommit=True)

# CRIA TABELA QTDADE VOTACAO MES

query = text("""
    DROP TABLE IF EXISTS {0}total_votacao_mes;
    CREATE TABLE {0}total_votacao_mes AS
        SELECT date_trunc('month', datavotacao_trunc)  as mes,
               SUM(numero_votacao) as numero_votacao
        FROM desenv.cam_total_votacao_dia
        GROUP BY mes
        ORDER BY mes DESC""".format(SCHEMA))
conn.execute(query, autocommit=True)

# CRIA TABELA QTDADE VOTACAO ANO
query = text("""
    DROP TABLE IF EXISTS {0}total_votacao_ano;
    CREATE TABLE {0}total_votacao_ano AS
        SELECT date_trunc('year', datavotacao_trunc)  as ano,
               SUM(numero_votacao) as numero_votacao
        FROM desenv.cam_total_votacao_dia
        GROUP BY ano
        ORDER BY ano DESC""".format(SCHEMA))
conn.execute(query, autocommit=True)

#### GERA VOTACAO BANCADA POR PARTIDO

In [8]:
votacao_bancada = pd.read_sql_table('votacao_bancada', conn, schema='a_camdep')
votacao_bancada.head()

,codproposicao,tipo,numero,ano,codsessao,datavotacao,sigla,orientacao
0,485758,MPV,511,2010,4251,2011-04-05 17:02:00,PT,Não
1,113717,PEC,41,2003,835,2003-09-10 18:52:00,PT,Não
2,102704,MPV,80,2002,737,2003-02-26 19:14:00,PCdoB,Sim
3,368170,MPV,392,2007,3305,2007-11-06 20:28:00,PV,Sim
4,357094,PEC,98,2007,4487,2011-11-29 20:10:00,PrPtdobPrpPhsPtcPsl,Liberado


In [9]:
# FILTRA SIGLA PARA SEPARAR POR PARTIDOS
import re

def check_partidos_blocos(sigla):
    """Checa se sigla está na lista de blocos ou partidos"""
    if sigla.upper() in partidos:
        return True
        
    elif sigla.upper() in blocos:
        return True
    
    else:
        return False

def substitui_sigla(sigla):
    
    for final, values in relacoes.items():
        if sigla.upper() in values:
            return final
    else:
        return sigla.upper()
            
def filtra_sigla(sigla):
    """
    Processa as siglas para separar por partidos ou blocos
    """
    sigla = sigla.strip().replace('Repr.', '').strip('.')
    
    if '/' in sigla:
        final = []
        for s in sigla.split('/'):
            if not check_partidos_blocos(s):
                print('Partido {} não está na lista'.format(s))
            else:
                final.append(substitui_sigla(s))
        return(final)
    
    elif check_partidos_blocos(sigla):
        return([substitui_sigla(sigla)])
    
    else:
        siglas = re.findall('[A-Z][^A-Z]*', sigla)
        
        if 'B' in siglas:
            siglas.remove('B')
            if 'Cdo' in siglas:
                siglas.append('PCdoB')
                siglas.remove('Cdo')
                siglas.remove('P')
            elif 'Tdo' in siglas:
                siglas.append('PTdoB')
                siglas.remove('Tdo')
                siglas.remove('P')
            elif 'Ptdo' in siglas:
                siglas.append('PTdoB')
                siglas.remove('Ptdo')
        
        final=[]
        for s in siglas:
            if not check_partidos_blocos(s):
                print('Partido {} não está na lista'.format(s))
            else:
                final.append(substitui_sigla(s))
        return(final)

In [10]:
j = 0
for i, row in tqdm.tqdm(votacao_bancada.iterrows()):
    
    row = row.to_dict()
    for sigla in filtra_sigla(row['sigla']):
        
        row['partido_bloco'] = sigla
        
        pickle.dump(row, open('temp/temp{}.p'.format(j), 'wb'))
        j += 1

37700it [00:13, 2726.00it/s]


In [11]:
votacao_bancada_partidos = []
for i in tqdm.tqdm(glob.glob('temp/*.p')):
    a = pickle.load(open(i, 'rb'))
    votacao_bancada_partidos.append(a)
    os.remove(i)
votacao_bancada_partidos = pd.DataFrame(votacao_bancada_partidos)

100%|██████████| 50690/50690 [00:07<00:00, 6664.66it/s]


In [12]:
votacao_bancada_partidos['votacao_id'] = votacao_bancada_partidos[['codproposicao', 
                                                       'codsessao' ,
                                                       'datavotacao']].apply(lambda x: 
                                                                             hash(tuple(x)),
                                                                             axis = 1)
votacao_bancada_partidos.to_sql('votacao_bancada_partidos', conn, schema='a_camdep', if_exists='replace')
votacao_bancada_partidos.head()

,ano,codproposicao,codsessao,datavotacao,numero,orientacao,partido_bloco,sigla,tipo,votacao_id
0,2016,2121843,16206,2017-05-15 21:01:00,757,Liberado,REDE,Repr.REDE,MPV,-2762957986797302247
1,2004,159554,942,2004-05-19 17:04:00,179,Não,PTB,PTB,MPV,-3775565451622503749
2,2015,1150196,5580,2015-06-24 18:11:00,672,Sim,PSB,PSB,MPV,-2263084988558699622
3,1997,25533,498,2000-06-27 18:18:00,3310,Não,PCDOB,PSB/PCDOB,PL,5813241135799075411
4,2009,434563,3798,2009-08-18 17:38:00,462,Não,PRB,PsbPCdoBPmnPrb,MPV,3936614211316915899


##### CRIA TABELA DO HASH DA PROPOSICAO

In [13]:
votacao_id_table = pd.read_sql_query('SELECT * FROM a_camdep.votacao_bancada_partidos',
                                        conn)

In [14]:
votacao_id_table.head()

,index,ano,codproposicao,codsessao,datavotacao,numero,orientacao,partido_bloco,sigla,tipo,votacao_id
0,0,2016,2121843,16206,2017-05-15 21:01:00,757,Liberado,REDE,Repr.REDE,MPV,-2762957986797302247
1,1,2004,159554,942,2004-05-19 17:04:00,179,Não,PTB,PTB,MPV,-3775565451622503749
2,2,2015,1150196,5580,2015-06-24 18:11:00,672,Sim,PSB,PSB,MPV,-2263084988558699622
3,3,1997,25533,498,2000-06-27 18:18:00,3310,Não,PCDOB,PSB/PCDOB,PL,5813241135799075411
4,4,2009,434563,3798,2009-08-18 17:38:00,462,Não,PRB,PsbPCdoBPmnPrb,MPV,3936614211316915899


In [15]:
votacao_id_table = votacao_id_table.drop_duplicates(subset=['votacao_id'])
votacao_id_table = votacao_id_table[['codsessao', 'datavotacao', 'codproposicao', 'votacao_id']]
votacao_id_table.index = votacao_id_table['votacao_id']
del votacao_id_table['votacao_id']

In [16]:
votacao_id_table.to_sql('id_proposicao_com_orient_governo', conn, schema='desenv', if_exists='replace')

#### CRIA TABELA ORIENTACAO GOVERNO

In [17]:
query = text("""
        DROP TABLE IF EXISTS {0}votacao_bancada_governo;
        CREATE TABLE {0}votacao_bancada_governo AS
            SELECT *
        FROM a_camdep.votacao_bancada_partidos
        WHERE partido_bloco = 'GOV'
        """.format(SCHEMA))
conn.execute(query, autocommit=True)

#### ATUALIZA TABELA DA VOTACAO DOS DEPUTADOS COM OS PARTIDOS CERTOS

#### GERA TABELA VOTACAO PARTIDO COM GOVERNO

In [18]:
# puxa tabela do sql
fidelidade_partido = pd.read_sql_query('SELECT * FROM a_camdep.votacao_bancada_partidos ORDER BY datavotacao DESC'.format(SCHEMA),
                                       conn)
# joga fora as entradas duplicadas por id e partido/bloco
fidelidade_partido = fidelidade_partido.drop_duplicates(subset=['votacao_id', 'partido_bloco'])
# reshape a tabela de fidelidade para ficar votacao_id x partidos
votacao_partido = fidelidade_partido.pivot(index='votacao_id', columns='partido_bloco', values='orientacao')
# seleciona a posicao do governo e exclui ela da tabela geral
votacao_gov = votacao_partido['GOV']
del votacao_partido['GOV']
# cria tabela verdade de 0 e 1 para cada votacao para cada partido
votacao_partido = votacao_partido.apply(lambda partidos: partidos == votacao_gov)
def bin(x):
    if x:
        return 1
    else:
        return 0
votacao_partido = votacao_partido.applymap(bin)
# junta tabela de id com votacao partido para ter info das datas e proposicao
votacao_partido.reset_index(level=0, inplace=True)


# upload para sql
votacao_partido.rename(columns=lambda x: x.lower(), inplace=True)
votacao_partido.to_sql('votacao_governo_partidos_dia', conn, schema='desenv', if_exists='replace')

#### GERA TABELA VOTACAO DEPUTADO COM GOVERNO


In [19]:
# puxa tabela do sql
fidelidade_deputado_raw = pd.read_sql_query('SELECT * FROM a_camdep.votacao_deputado ORDER BY datavotacao DESC'.format(SCHEMA),
                                        conn)
vot_governo = pd.read_sql_query('SELECT * FROM desenv.cam_votacao_bancada_governo',
                                        conn)

In [20]:
vot_governo = vot_governo[['codproposicao', 'codsessao', 'datavotacao', 'orientacao', 'votacao_id']]

In [21]:
# merge hash para cada votacao na tabela de fidelidade
fidelidade_deputado = fidelidade_deputado_raw.merge(vot_governo,
                                                on=['codproposicao', 'codsessao', 'datavotacao'])

# deleta entradas duplicadas 
fidelidade_deputado = fidelidade_deputado.drop_duplicates(subset=['votacao_id', 'idecadastro'])

# deleta entradas que o governo não se posicionou
fidelidade_deputado = fidelidade_deputado.dropna(subset=['orientacao'])

# deleta entradas que o governo se posicionou como 'Liberado'
fidelidade_deputado = fidelidade_deputado[fidelidade_deputado['orientacao'] != 'Liberado']

# deleta entradas que o governo se posicionou como 'Abstenção'
fidelidade_deputado = fidelidade_deputado[fidelidade_deputado['orientacao'] != 'Abstenção']

# deleta entradas com voto parlamentar  '-'
fidelidade_deputado = fidelidade_deputado[fidelidade_deputado['voto'] != '-']

# deleta entradas com voto parlamentar  'Art. 17'
fidelidade_deputado = fidelidade_deputado[fidelidade_deputado['voto'] != 'Art. 17']

# cria coluna de apoio
fidelidade_deputado['fidelidade'] = np.where(fidelidade_deputado['orientacao'] 
                                             == fidelidade_deputado['voto'],
                                            1, 0)
# cria coluna para quorum
fidelidade_deputado = fidelidade_deputado.merge(fidelidade_deputado.groupby('votacao_id').count()['codproposicao'].to_frame().reset_index(), on='votacao_id')
fidelidade_deputado.rename(columns={'codproposicao_y': 'quorum'}, inplace=True)

# cria coluna partido atualizado
fidelidade_deputado['partido_atualizado'] = fidelidade_deputado['partido'].apply(substitui_sigla)

# cria coluna do tamanho da bancada na votação
tam_bancada = fidelidade_deputado.groupby(['votacao_id', 'partido_atualizado']).count().reset_index()[['votacao_id','partido_atualizado', 'tipo']]
fidelidade_deputado = fidelidade_deputado.merge(tam_bancada, on=['votacao_id', 'partido_atualizado'])
fidelidade_deputado.rename(columns={'tipo_y': 'tam_bancada'}, inplace=True)

# prepara e up no banco
fidelidade_deputado.rename(columns=lambda x: x.lower(), inplace=True)

In [22]:
fidelidade_deputado.to_sql('votacao_governo_deputados_dia',
                           conn, 
                           schema='desenv', 
                           if_exists='replace',
                           chunksize=10000)